<a href="https://colab.research.google.com/github/manvgom/Pickioset/blob/main/SortOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -rf SortOutput
!mkdir SortOutput
!mkdir /content/SortOutput/dataset
!mkdir /content/SortOutput/dataset/kds_dataset
!mkdir /content/SortOutput/dataset/coco_classes
!mkdir /content/SortOutput/dataset/configuration
!cp '/content/drive/MyDrive/MTDI_TFM/Labeling.zip' '/content/SortOutput'
!unzip '/content/SortOutput/Labeling.zip' -d '/content/SortOutput'
!rm -rf /content/SortOutput/Labeling.zip /content/SortOutput/__MACOSX

import os
dirlist = os.listdir('/content/SortOutput/Labeling/true_dataset_nevera')
dirlist.remove('.DS_Store')
dirlist

for dirname in dirlist:
  %cp /content/SortOutput/Labeling/true_dataset_nevera/{dirname}/* /content/SortOutput/dataset/kds_dataset

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/SortOutput/Labeling/true_dataset_nevera/Video_nevera_05_mp4/Video_nevera_05_mp4_90.jpg  
  inflating: /content/SortOutput/__MACOSX/Labeling/true_dataset_nevera/Video_nevera_05_mp4/._Video_nevera_05_mp4_90.jpg  
  inflating: /content/SortOutput/Labeling/true_dataset_nevera/Video_nevera_05_mp4/Video_nevera_05_mp4_84.jpg  
  inflating: /content/SortOutput/__MACOSX/Labeling/true_dataset_nevera/Video_nevera_05_mp4/._Video_nevera_05_mp4_84.jpg  
  inflating: /content/SortOutput/Labeling/true_dataset_nevera/Video_nevera_05_mp4/Video_nevera_05_mp4_53.jpg  
  inflating: /content/SortOutput/__MACOSX/Labeling/true_dataset_nevera/Video_nevera_05_mp4/._Video_nevera_05_mp4_53.jpg  
  inflating: /content/SortOutput/Labeling/true_dataset_nevera/Video_nevera_05_mp4/Video_nevera_05_mp4_47.jpg  
  inflating: /content/SortOutput/__MACOSX/Labeling/true_dataset_nevera/Video_nevera_05_mp4/._Video_nevera_05_mp4_47.jpg  
  infla

In [3]:
import glob
with open(glob.glob('/content/SortOutput/Labeling/*.txt')[0]) as f:
    classlist = f.read().splitlines()

while '' in classlist:
  classlist.remove('')
  
coco_classes = str(classlist)
coco_classes = coco_classes.replace("[", "(")
coco_classes = coco_classes.replace("]", ")")
coco_classes = 'COCO_CLASSES ='+coco_classes

with open('/content/SortOutput/dataset/coco_classes/coco_classes.py', 'w') as f:
    f.write(coco_classes)

In [4]:
!git clone https://github.com/manvgom/pickioConfig.git
!mv /content/pickioConfig//nano.py /content/SortOutput/dataset/configuration
!rm -rf pickioConfig/

Cloning into 'pickioConfig'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [84]:
epochs = 15
classnumber = len(classlist)
term1 = 'self.num_classes ='
term2 = 'self.max_epoch ='
filepath = '/content/SortOutput/dataset/configuration/nano.py'

#Buscar lineas relacionadas con las clases y los epoch
t_loss=[]
with open(filepath) as fp:
  for line in reversed(list(fp)):
    if term1 in line:
      t_loss.append(str(line))
    if term2 in line:
      t_loss.append(str(line))

#Arreglar formato en las lineas seleccionadas
count=0
while count < len(t_loss):
  t_loss[count] = t_loss[count].lstrip()
  t_loss[count] = t_loss[count].rstrip()
  t_loss[count] = t_loss[count].strip('\n')
  count+=1
t_loss[1]

# Leer archivo 
with open(filepath, 'r') as file :
  filedata = file.read()

# Cambiar configuración
filedata = filedata.replace(t_loss[0], 'self.max_epoch = ' + str(epochs))
filedata = filedata.replace(t_loss[1], 'self.num_classes =' + str(classnumber))

# Guradar configuración
with open(filepath, 'w') as file:
  file.write(filedata)